In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.4 MB/s 
     |████████████████████████████████| 2.9 MB 11.8 MB/s 
     |████████████████████████████████| 3.3 MB 31.5 MB/s 
     |████████████████████████████████| 1.2 MB 41.5 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 52.9 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=6b067e60fd2a19e8e66b1aae2460303985c0f2bf20d632fcbba493f466de0b1e
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Manual testing:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('uclanlp/visualbert-nlvr2-coco-pre')
embeddings = model.encode(sentences)
print(embeddings.shape, embeddings[0, :10])

DEBUG:filelock:Attempting to acquire lock 140260419496016 on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/.gitattributes.lock
DEBUG:filelock:Lock 140260419496016 acquired on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/.gitattributes.lock


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140260419496016 on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/.gitattributes.lock
DEBUG:filelock:Lock 140260419496016 released on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/.gitattributes.lock
DEBUG:filelock:Attempting to acquire lock 140260419495952 on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/config.json.lock
DEBUG:filelock:Lock 140260419495952 acquired on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/config.json.lock


Downloading:   0%|          | 0.00/631 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140260419495952 on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/config.json.lock
DEBUG:filelock:Lock 140260419495952 released on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/config.json.lock
DEBUG:filelock:Attempting to acquire lock 140260455922064 on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/pytorch_model.bin.lock
DEBUG:filelock:Lock 140260455922064 acquired on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/pytorch_model.bin.lock


Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140260455922064 on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/pytorch_model.bin.lock
DEBUG:filelock:Lock 140260455922064 released on /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre/pytorch_model.bin.lock
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/uclanlp_visualbert-nlvr2-coco-pre were not used when initializing VisualBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing VisualBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a B

KeyError: ignored

In [ ]:
# Automated
from collections import OrderedDict

from transformers.models.auto.tokenization_auto import TOKENIZER_MAPPING_NAMES
from transformers.models.auto.modeling_auto import MODEL_MAPPING_NAMES
from transformers.models.auto.configuration_auto import CONFIG_MAPPING_NAMES, CONFIG_ARCHIVE_MAP_MAPPING_NAMES

from transformers import RobertaTokenizer

from sentence_transformers import SentenceTransformer

SE_COMP_MAPPING = OrderedDict()
sentences = ["This is an example sentence", "Each sentence is converted"]
!wget https://huggingface.co/bert-base-uncased/blob/main/vocab.txt

# Skip models that will crash the session
INCOMPATIBLE_AND_TOO_BIG = ["blenderbot"]

for model_type, model_name in MODEL_MAPPING_NAMES.items():
    print("RUNNING: ", model_type, model_name)
    if model_type in INCOMPATIBLE_AND_TOO_BIG:
        continue
    if isinstance(model_name, tuple):
        print(f"Choosing {model_name[0]} from {model_name}")
        model_name = model_name[0]
    # Incompatible as SentenceTransformers uses AutoTokenizer to load the model
    if (model_type not in TOKENIZER_MAPPING_NAMES) or (model_type not in CONFIG_MAPPING_NAMES):
        SE_COMP_MAPPING[model_type] = False
        continue

    config_name = CONFIG_MAPPING_NAMES[model_type]
    config = getattr(__import__("transformers", fromlist=[config_name]), config_name)
    model = getattr(__import__("transformers", fromlist=[model_name]), model_name)

    tokenizer_name = TOKENIZER_MAPPING_NAMES[model_type]
    if isinstance(tokenizer_name, tuple):
        tokenizer_name = tokenizer_name[0]
    tokenizer = getattr(__import__("transformers", fromlist=[tokenizer_name]), tokenizer_name)
    try:
        # Load model as small as possible else Memory error e.g. for GPT-J
        base_model = model(config(n_layer=1, layers=1, num_layers=1, num_hidden_layers=1, attention_types=[[["global"], 1]], encoder_layers=1, decoder_layers=1))
        base_model.save_pretrained("./model")
        #base_tok = tokenizer("./vocab.txt")
        #ase_tok.save_pretrained("./model")

        # RobertaTokenizer will work for most
        base_tok = RobertaTokenizer.from_pretrained("roberta-base")
        base_tok.save_pretrained("./model")
        sen_model = SentenceTransformer("./model")
        embeddings = sen_model.encode(sentences)

        print(f"Output for {model_type}: {embeddings.shape}")
        !rm -r ./model
        del base_model
        del sen_model
        SE_COMP_MAPPING[model_type] = True

    except Exception as e1:
        print(f"EXCEPTING once due to {e1}")
        # Else try its own tokenizer
        try:
            !rm ./model/tokenizer_config.json
            !rm ./model/special_tokens_map.json
            !rm ./model/vocab.json
            !rm ./model/merges.txt
            !rm ./model/added_tokens.json

            # Else try the models tokenizer - but possibly with wrong initialization
            base_tok = tokenizer("./vocab.txt")
            base_tok.save_pretrained("./model")

            sen_model = SentenceTransformer("./model")
            embeddings = sen_model.encode(sentences)

            print(f"Output for {model_type}: {embeddings.shape}")
            !rm -r ./model
            del base_model
            del sen_model
            SE_COMP_MAPPING[model_type] = True
        
        except Exception as e2:
            print(f"EXCEPTING twice due to {e2}")
            # Else try pretrained model
            try: 
                config_archive_name = CONFIG_ARCHIVE_MAP_MAPPING_NAMES[model_type]
                config_archive = getattr(__import__("transformers", fromlist=[config_archive_name]), config_archive_name)
                base_weight = list(config_archive.keys())[0]
                sen_model = SentenceTransformer(base_weight)
                embeddings = sen_model.encode(sentences)
                print(f"Output for {model_type}: {embeddings.shape}")
                !rm -r ./model
                del base_model
                del sen_model
                SE_COMP_MAPPING[model_type] = True
            except Exception as e3:
                print(f"INCOMPATIBLE {model_type} due to {e1} and {e2} and {e3}")
                SE_COMP_MAPPING[model_type] = False
# Mistakes:
# gpt_neo is incompatible (incompatible due to the gpt-neo tokenizer not having a padding token)
# sentence-transformer asks for the padding token internally; With a RobertaTokenizer like here it would work

--2021-10-26 12:38:53--  https://huggingface.co/bert-base-uncased/blob/main/vocab.txt
Resolving huggingface.co (huggingface.co)... 3.94.225.255, 34.197.18.55, 2600:1f18:147f:e850:2b5a:d015:5266:6170, ...
Connecting to huggingface.co (huggingface.co)|3.94.225.255|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6719789 (6.4M) [text/html]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>]   6.41M  22.6MB/s    in 0.3s    

2021-10-26 12:38:54 (22.6 MB/s) - ‘vocab.txt’ saved [6719789/6719789]

RUNNING:  fnet FNetModel


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/262k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140260451325840 on /root/.cache/torch/sentence_transformers/google_tapas-base-finetuned-sqa/vocab.txt.lock
DEBUG:filelock:Lock 140260451325840 released on /root/.cache/torch/sentence_transformers/google_tapas-base-finetuned-sqa/vocab.txt.lock


INCOMPATIBLE tapas due to 
TapasModel requires the torch-scatter library but it was not found in your environment. You can install it with pip as
explained here: https://github.com/rusty1s/pytorch_scatter.
 and Can't load config for './model'. Make sure that:

- './model' is a correct model identifier listed on 'https://huggingface.co/models'

- or './model' is the correct path to a directory containing a config.json file

 and 
TapasModel requires the torch-scatter library but it was not found in your environment. You can install it with pip as
explained here: https://github.com/rusty1s/pytorch_scatter.

RUNNING:  ibert IBertModel


Output for ibert: (2, 768)
RUNNING:  splinter SplinterModel


Output for splinter: (2, 768)


In [ ]:
SE_COMP_MAPPING

OrderedDict([('fnet', False),
             ('gptj', False),
             ('layoutlmv2', False),
             ('beit', False),
             ('rembert', True),
             ('visual_bert', False),
             ('canine', True),
             ('roformer', True),
             ('clip', False),
             ('bigbird_pegasus', True),
             ('deit', False),
             ('luke', True),
             ('detr', False),
             ('gpt_neo', True),
             ('big_bird', True),
             ('speech_to_text', False),
             ('vit', False),
             ('wav2vec2', False),
             ('hubert', False),
             ('m2m_100', False),
             ('convbert', True),
             ('led', True),
             ('blenderbot-small', False),
             ('retribert', False),
             ('mt5', False),
             ('t5', False),
             ('pegasus', False),
             ('marian', False),
             ('mbart', True),
             ('distilbert', True),
             ('albert', 

In [ ]:
for k, v in sorted(SE_COMP_MAPPING.items()):
    if v:
      print(f'"{k}",')

"albert",
"bart",
"bert",
"bert-generation",
"big_bird",
"bigbird_pegasus",
"camembert",
"canine",
"convbert",
"ctrl",
"deberta",
"deberta-v2",
"distilbert",
"electra",
"flaubert",
"fsmt",
"funnel",
"gpt2",
"gpt_neo",
"ibert",
"layoutlm",
"led",
"longformer",
"luke",
"mbart",
"mobilebert",
"mpnet",
"openai-gpt",
"rembert",
"roberta",
"roformer",
"splinter",
"squeezebert",
"xlm",
"xlm-roberta",
